In [9]:
import urllib3
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
from io import StringIO
import json
from headers import *
import controllers

import importlib
import models
importlib.reload(controllers)
importlib.reload(models)
controller = controllers.FuturesDataController()
model = models.FuturesDataModel()



In [3]:
controller.crawler_taifex_data("2024/09/08")

爬取日期: 2024/09/09
爬取成功
轉換成功


/Users/hayden/Documents/pyDB/controllers.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filter[col] = pd.to_numeric(df_filter[col].astype(str).str.replace(',', ''), errors='coerce')
/Users/hayden/Documents/pyDB/controllers.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filter[col] = pd.to_numeric(df_filter[col].astype(str).str.replace(',', ''), errors='coerce')
/Users/hayden/Documents/pyDB/controllers.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a D

In [4]:
df

,商品名稱,身份別,多方交易口數,多方交易金額,空方交易口數,空方交易金額,交易淨口數,交易淨金額,多方未平倉口數,多方未平倉金額,空方未平倉口數,空方未平倉金額,未平倉淨口數,未平倉淨金額
0,臺股期貨,自營商,13246,55601062,11754,49368888,1492,6232174,5568,23477208,4400,18552000,1168,4925208
1,臺股期貨,投信,492,2075569,328,1384444,164,691125,32120,135437089,8815,37169321,23305,98267768
2,臺股期貨,外資,117414,492754797,117922,495218445,-508,-2463649,18504,78022810,46601,196495997,-28097,-118473187
9,小型臺指期貨,自營商,22117,23207247,22912,24067279,-795,-860032,2663,2807641,12679,13365228,-10016,-10557587
10,小型臺指期貨,投信,4,4190,0,0,4,4190,149,157068,17,17921,132,139147
11,小型臺指期貨,外資,214661,225365024,213317,224118975,1344,1246049,5880,6198384,2985,3146684,2895,3051700
12,微型臺指期貨,自營商,24126,5067995,20690,4355071,3436,712924,4078,859745,8181,1724672,-4103,-864927
13,微型臺指期貨,投信,0,0,0,0,0,0,0,0,0,0,0,0
14,微型臺指期貨,外資,85837,18026205,89560,18821540,-3723,-795335,22,4638,10540,2222147,-10518,-2217509


In [5]:
date = "2024/09/09"
data = {
            "date":datetime.strptime(date,"%Y/%m/%d"),
            "期貨契約":[]
        }
for _, row in df.iterrows():
    contract = {
        "商品名稱": row['商品名稱'],
        "身份別": row['身份別'],
        "多方交易口數": int(row['多方交易口數']),
        "多方交易金額": int(row['多方交易金額']),
        "空方交易口數": int(row['空方交易口數']),
        "空方交易金額": int(row['空方交易金額']),
        "交易淨口數": int(row['交易淨口數']),
        "交易淨金額": int(row['交易淨金額']),
        "多方未平倉口數": int(row['多方未平倉口數']),
        "多方未平倉金額": int(row['多方未平倉金額']),
        "空方未平倉口數": int(row['空方未平倉口數']),
        "空方未平倉金額": int(row['空方未平倉金額']),
        "未平倉淨口數": int(row['未平倉淨口數']),
        "未平倉淨金額": int(row['未平倉淨金額'])
    }
    data["期貨契約"].append(contract)
    

In [6]:
print(data)

{'date': datetime.datetime(2024, 9, 9, 0, 0), '期貨契約': [{'商品名稱': '臺股期貨', '身份別': '自營商', '多方交易口數': 13246, '多方交易金額': 55601062, '空方交易口數': 11754, '空方交易金額': 49368888, '交易淨口數': 1492, '交易淨金額': 6232174, '多方未平倉口數': 5568, '多方未平倉金額': 23477208, '空方未平倉口數': 4400, '空方未平倉金額': 18552000, '未平倉淨口數': 1168, '未平倉淨金額': 4925208}, {'商品名稱': '臺股期貨', '身份別': '投信', '多方交易口數': 492, '多方交易金額': 2075569, '空方交易口數': 328, '空方交易金額': 1384444, '交易淨口數': 164, '交易淨金額': 691125, '多方未平倉口數': 32120, '多方未平倉金額': 135437089, '空方未平倉口數': 8815, '空方未平倉金額': 37169321, '未平倉淨口數': 23305, '未平倉淨金額': 98267768}, {'商品名稱': '臺股期貨', '身份別': '外資', '多方交易口數': 117414, '多方交易金額': 492754797, '空方交易口數': 117922, '空方交易金額': 495218445, '交易淨口數': -508, '交易淨金額': -2463649, '多方未平倉口數': 18504, '多方未平倉金額': 78022810, '空方未平倉口數': 46601, '空方未平倉金額': 196495997, '未平倉淨口數': -28097, '未平倉淨金額': -118473187}, {'商品名稱': '小型臺指期貨', '身份別': '自營商', '多方交易口數': 22117, '多方交易金額': 23207247, '空方交易口數': 22912, '空方交易金額': 24067279, '交易淨口數': -795, '交易淨金額': -860032, '多方未平倉口數': 2663, '多方未平倉金額': 2807641, '空方未平倉口數': 

In [10]:
client = model.connect_mongo()

成功連接到 MongoDB!


In [11]:
db = client["findata"]
collection = db["txf_oi"]


In [12]:
existing_data = collection.find_one({'date':data['date']})

In [15]:
print(existing_data)
print(data)

None
{'date': datetime.datetime(2024, 9, 9, 0, 0), '期貨契約': [{'商品名稱': '臺股期貨', '身份別': '自營商', '多方交易口數': 13246, '多方交易金額': 55601062, '空方交易口數': 11754, '空方交易金額': 49368888, '交易淨口數': 1492, '交易淨金額': 6232174, '多方未平倉口數': 5568, '多方未平倉金額': 23477208, '空方未平倉口數': 4400, '空方未平倉金額': 18552000, '未平倉淨口數': 1168, '未平倉淨金額': 4925208}, {'商品名稱': '臺股期貨', '身份別': '投信', '多方交易口數': 492, '多方交易金額': 2075569, '空方交易口數': 328, '空方交易金額': 1384444, '交易淨口數': 164, '交易淨金額': 691125, '多方未平倉口數': 32120, '多方未平倉金額': 135437089, '空方未平倉口數': 8815, '空方未平倉金額': 37169321, '未平倉淨口數': 23305, '未平倉淨金額': 98267768}, {'商品名稱': '臺股期貨', '身份別': '外資', '多方交易口數': 117414, '多方交易金額': 492754797, '空方交易口數': 117922, '空方交易金額': 495218445, '交易淨口數': -508, '交易淨金額': -2463649, '多方未平倉口數': 18504, '多方未平倉金額': 78022810, '空方未平倉口數': 46601, '空方未平倉金額': 196495997, '未平倉淨口數': -28097, '未平倉淨金額': -118473187}, {'商品名稱': '小型臺指期貨', '身份別': '自營商', '多方交易口數': 22117, '多方交易金額': 23207247, '空方交易口數': 22912, '空方交易金額': 24067279, '交易淨口數': -795, '交易淨金額': -860032, '多方未平倉口數': 2663, '多方未平倉金額': 2807641, '空方未平倉

In [17]:
result = collection.insert_one(data)

In [ ]:
df = controller.crawler_twse_total_return("20240909")
df

In [ ]:
df2 = controller.crawler_twse_data("20240909")
df2

In [2]:
params ={
    'response':'json',
    'date':"20240810"
}
rsp = requests.get("https://www.twse.com.tw/rwd/zh/TAIEX/MFI94U",data=params,headers=TWSE_DOWN)

In [ ]:
d2 = json.loads(rsp.text)
d2

In [ ]:
df2 = pd.DataFrame(d2['data'],columns=['date','tri_close'])
df2

In [ ]:
df2.iloc[:,1:] = df2.iloc[:,1:].apply(lambda x: x.str.replace(',','').astype(float), axis=1)
df2

In [ ]:
df2[['year','month','day']] = df2['date'].str.split('/',expand=True)
df2['year'] = df2['year'].astype(int)+1911
df2['date'] = df2['year'].astype(str)+'-'+df2['month'].str.zfill(2) + "-" + df2['day'].str.zfill(2)
df2['date'] = pd.to_datetime(df2['date'])
df2

In [ ]:
#tdf = controller.crawler_twse_data("20240801")
xdf = controller.crawler_twse_data("20240801")

In [ ]:
xdf

In [ ]:
date = "20240905"
url = "https://www.twse.com.tw/indicesReport/MI_5MINS_HIST"
params = {
    'response':'json',
    'date':f'{date}'
}

response = requests.get(url,data=params,headers=TWSE_DOWN)


In [ ]:
response.text

In [ ]:
data = json.loads(response.text)
data

In [ ]:
df = pd.DataFrame(data['data'],columns=['date','open','high','low','close'])
df

In [ ]:
df.iloc[:, 1:] = df.iloc[:, 1:].apply(lambda x: x.str.replace(',', '').astype(float), axis=1)

In [ ]:
df

In [ ]:
df[['year', 'month', 'day']] = df['date'].str.split('/', expand=True)
df['year'] = df['year'].astype(int) + 1911
df['date'] = df['year'].astype(str) + '-' + df['month'].str.zfill(2) + '-' + df['day'].str.zfill(2)
df

In [ ]:
df[['year','Month', 'Day']] = df['Date'].str.split("/",expand=True,)


In [ ]:
df

In [ ]:
df['Year'] = df['Year'].astype(int) + 1911


In [ ]:
df

In [ ]:
df['Date']  = (pd.to_datetime(df['Year'].astype(str) + '/' +
df['Month'].astype(str) + '/' +
df['Day'].astype(str)))

In [ ]:
df

In [ ]:
df = df.iloc[:, :-3]







In [ ]:
df